In [ ]:


from typing import Literal

def count_tokens(
    text: str,
    provider: Literal["openai", "anthropic"],
    *,
    model: str | None = None,
) -> int:
    """
    Return the number of tokens `text` would use with the chosen provider.

    Parameters
    ----------
    text      : The string you want to measure.
    provider  : "openai" or "anthropic".
    model     : (optional) A specific model name.
                • OpenAI – defaults to "gpt-3.5-turbo".
                • Anthropic – only needed if you plan to call the
                  beta messages‐style counter.

    Notes
    -----
    • OpenAI token rules differ by model family; we pick the correct
      encoding automatically and fall back to ``cl100k_base`` if the
      model is unknown. :contentReference[oaicite:0]{index=0}  
    • Anthropic’s Python SDK exposes a simple ``client.count_tokens()``
      helper for plain strings and a richer
      ``client.beta.messages.count_tokens()`` for structured prompts. :contentReference[oaicite:1]{index=1}
    """
    provider = provider.lower()

    if provider == "openai":
        import tiktoken

        model = model or "gpt-3.5-turbo"
        try:
            enc = tiktoken.encoding_for_model(model)   # provider-specific BPE
        except KeyError:                               # unknown model → default
            enc = tiktoken.get_encoding("cl100k_base")
        return len(enc.encode(text))

    elif provider == "anthropic":
        from anthropic import Anthropic

        client = Anthropic()           # uses ANTHROPIC_API_KEY if set
        return client.messages.count_tokens(
            model="claude-3-7-sonnet-20250219",
            messages=[{"role": "user", "content": text}]
        ).input_tokens
        

    else:
        raise ValueError("provider must be 'openai' or 'anthropic'")


In [ ]:
count_tokens("Test", provider="anthropic")

In [ ]:
count_tokens(StudentAgent().get_output_jsonschema().__str__(), provider="anthropic")
count_tokens(StudentAgent().get_output_jsonschema().__str__(), provider="openai")

In [ ]:
mllm.config.default_models.normal